In [1]:
from IPython.core.display import HTML
with open('../style.css', 'r') as file:
    css = file.read()
HTML(css)

# Saving the Infidels

In this notebook we want so solve a famous search problem, which is usually known as the
[missionaries and cannibals problem](https://en.wikipedia.org/wiki/Missionaries_and_cannibals_problem):
Three missinaries and three infidels have to cross a river in order to get to a church where the infidels can be baptized.  In order to cross the river, they have to take a small boat that can take at most two passengers.  If at any moments at any shore there are more infidels than missionaries, then the missionaries have a problem, since the infidels have a diet that includes human flesh.

We will encode this problem as a *constraint satisfaction problem*.  In order to do so, we assume that the
problem can be solved with $n\in\mathbb{N}$ crossing of the river.  We use the following variables:
* $\texttt{M}i$ for $i\in\{0,\cdots,n\}$ is the number of missionaries on the western shore after the 
  $i^{\textrm{th}}$ crossing.
* $\texttt{C}i$ for $i\in\{0,\cdots,n\}$ is the number of infidels on the western shore after the 
  $i^{\textrm{th}}$ crossing.
* $\texttt{B}i$ for $i\in\{0,\cdots,n\}$ is the number of boats on the western shore after the 
  $i^{\textrm{th}}$ crossing.

## Auxiliary Functions

The function `flatten` takes a list of lists `LoL` and returns a list containing all the elements contained in any of the lists in `LoL`.

In [2]:
def flatten(LoL):
    return [x for L in LoL for x in L]

In [3]:
flatten([[1,2], [3,4]])

[1, 2, 3, 4]

The function `start` takes three integers as input:
* `M` is the number of missionaries on the western shore,
* `C` is the number of infidels on the western shore,

It returns `True` in the initial state where everybody is on the western shore.

In [4]:
def start(M, C, B):
    return M == 3 and C == 3 and B == 1

The function `goal` takes two integers as input:
* `M` is the number of missionaries on the western shore,
* `C` is the number of infidels on the western shore,

It returns `True` in the state where everybody is on the eastern shore
and hence nobody is left on the wetsern shore.

In [5]:
def goal(M, C):
    return M == 0 and C == 0

The function `invariant` takes three integers as input:
* `M` is the number of missionaries on the western shore,
* `C` is the number of infidels on the western shore,
* `B` is the number of boats on the western shore.

It returns `True` if there is no problem on either shore of the river.  
There is no problem if any of the following conditions is true:
* There are no missionaries on the western side of the shore, i.e. 
  $\texttt{M} = 0$.  
  Then all missionaries are on the eastern side of the shore.
* All missionaries are on the western side of the shore, i.e. $\texttt{M} = 3$.
  Then there are no missionaries on the eastern side of the shore.
* The number of missionaries on the western side is the same as the number of 
  infidels on that side, i.e. $\texttt{M} = \texttt{C}$.  Then the numbers of 
  missionaries and infidels have to match on the eastern shore as well.

In [6]:
def invariant(M, C):
    return M == 0 or M == 3 or M == C

The function `transition` takes 6 arguments:
* `M𝛼` is the number of missionaries on the eastern shore before the crossing.
* `C𝛼` is the number of infidels on the eastern shore before the crossing.
* `B𝛼` is the number of boats on the eastern shore before the crossing. 
* `M𝛽` is the number of missionaries on the eastern shore after the crossing.
* `C𝛽` is the number of infidels on the eastern shore after the crossing.
* `B𝛽` is the number of infidels on the eastern shore after the crossing.
returns a set of formulas that is true if the missionaries starting on one shore arrive at the opposite  shore after the $i^{\textrm{th}}$ crossing.  Note that if $i$ is odd, then during the $i^{\textrm{th}}$ crossing the boat travels from the western shore to the eastern shore.  If $i$ is even, the boat travels from the eastern shore to the western shore.

In [7]:
def transition(M𝛼, C𝛼, B𝛼, M𝛽, C𝛽, B𝛽):
    if not (0 <= B𝛼 <= 1 and B𝛽 == 1 - B𝛼):
        return False
    if B𝛼 == 1:
        return (1 <= M𝛼 - M𝛽 + C𝛼 - C𝛽 <= 2 and
                M𝛽 <= M𝛼 and C𝛽 <= C𝛼)
    else:
        return (1 <= M𝛽 - M𝛼 + C𝛽 - C𝛼 <= 2 and
                M𝛽 >= M𝛼 and C𝛽 >= C𝛼)

The function `change_ok(i)` returns a set of formulas that is true if the missionaries starting on one shore arrive at the opposite  shore after the $i^{\textrm{th}}$ crossing.  Note that if $i$ is odd, then during the $i^{\textrm{th}}$ crossing the boat travels from the western shore to the eastern shore.  If $i$ is even, the boat travels from the eastern shore to the western shore.

The function `missionaries_CSP` creates a CSP that tries to solve the problem with `n` crossings.

In [8]:
def missionaries_CSP(n):
    "Returns a CSP encoding the problem."
    Lists        = [[f'M{i}', f'C{i}', f'B{i}'] for i in range(n+1)]
    Variables    = flatten(Lists)
    Values       = { 0, 1, 2, 3 }
    Constraints  = { 'start(M0, C0, B0)'     }  # start state
    Constraints |= { f'goal(M{n}, C{n})' }  # goal state
    for i in range(n):
        Constraints.add(f'invariant(M{i}, C{i})')
        Constraints.add(f'transition(M{i}, C{i}, B{i}, M{i+1}, C{i+1}, B{i+1})')
    return Variables, Values, Constraints

In [9]:
missionaries_CSP(3)

(['M0', 'C0', 'B0', 'M1', 'C1', 'B1', 'M2', 'C2', 'B2', 'M3', 'C3', 'B3'],
 {0, 1, 2, 3},
 {'goal(M3, C3)',
  'invariant(M0, C0)',
  'invariant(M1, C1)',
  'invariant(M2, C2)',
  'start(M0, C0, B0)',
  'transition(M0, C0, B0, M1, C1, B1)',
  'transition(M1, C1, B1, M2, C2, B2)',
  'transition(M2, C2, B2, M3, C3, B3)'})

In [10]:
%run Backtracking-Constraint-Solver.ipynb

The function `find_solution` computes a solution to the problem of saving the infidels.

In [11]:
def find_solution():
    n = 1
    while True:
        print(n)
        CSP = missionaries_CSP(n)
        Solution = solve(CSP)
        if Solution != None:
            return n, Solution
        n += 2

On my desktop computer (2017 iMac with 3.4 GHz Quad-Core Intel i5) it takes about 2 seconds to solve the problem. 

In [12]:
%%time
n, Solution = find_solution()
n, Solution

1
3
5
7
9
11
CPU times: user 1.35 s, sys: 8.07 ms, total: 1.36 s
Wall time: 1.37 s


(11,
 {'M0': 3,
  'C0': 3,
  'B0': 1,
  'M1': 2,
  'C1': 2,
  'B1': 0,
  'M2': 3,
  'C2': 2,
  'B2': 1,
  'M3': 3,
  'C3': 0,
  'B3': 0,
  'M4': 3,
  'C4': 1,
  'B4': 1,
  'M5': 1,
  'C5': 1,
  'B5': 0,
  'M6': 2,
  'C6': 2,
  'B6': 1,
  'M7': 0,
  'C7': 2,
  'B7': 0,
  'M8': 0,
  'C8': 3,
  'B8': 1,
  'M9': 0,
  'C9': 1,
  'B9': 0,
  'M10': 0,
  'C10': 2,
  'B10': 1,
  'M11': 0,
  'C11': 0,
  'B11': 0})

In [13]:
def show_solution(Solution, n):
    for i in range(n+1):
        M = Solution[f'M{i}']
        C = Solution[f'C{i}']
        B = Solution[f'B{i}']
        print('😇' * M + '🥷' * C + ' ' * 28 + '😇' * (3 - M) + '🥷' * (3 - C))
        if B == 1:
            MB = Solution[f'M{i+1}'] - Solution[f'M{i}']
            CB = Solution[f'C{i+1}'] - Solution[f'C{i}']
            print(' ' * 12 + '>>> ' + '😇'*MB + '🥷'*CB + ' >>>')
        elif i + 1 < n:
            MB = Solution[f'M{i}'] - Solution[f'M{i+1}']
            CB = Solution[f'C{i}'] - Solution[f'C{i+1}']
            print(' ' * 12 + '<<< ' + '😇'*MB + '🥷'*CB + ' <<<')

In [14]:
show_solution(Solution, n)

😇😇😇🥷🥷🥷                            
            >>>  >>>
😇😇🥷🥷                            😇🥷
            <<<  <<<
😇😇😇🥷🥷                            🥷
            >>>  >>>
😇😇😇                            🥷🥷🥷
            <<<  <<<
😇😇😇🥷                            🥷🥷
            >>>  >>>
😇🥷                            😇😇🥷🥷
            <<<  <<<
😇😇🥷🥷                            😇🥷
            >>>  >>>
🥷🥷                            😇😇😇🥷
            <<<  <<<
🥷🥷🥷                            😇😇😇
            >>>  >>>
🥷                            😇😇😇🥷🥷
            <<<  <<<
🥷🥷                            😇😇😇🥷
            >>>  >>>
                            😇😇😇🥷🥷🥷
